### Invalid URL and URLLIP

In [1]:
# import necessary libraries
import os, csv
import shutil
import urllib
from urllib.request import urlopen
from socket import error as SocketError
import errno
import pandas as pd
import httplib2
from requests.auth import HTTPBasicAuth
import requests
from requests.auth import HTTPDigestAuth
from bs4 import BeautifulSoup
import re
from urllib.parse import urljoin

In [2]:
#setting directories
micro_sample_cvs = "/Users/anhnguyen/Desktop/research/scraping_Python/excel_files/micro-sample_MANUAL_Apr17.xlsx"
valid = "/Users/anhnguyen/Desktop/research/scraping_Python/text_files/valid_march.txt"
invalid = "/Users/anhnguyen/Desktop/research/scraping_Python/text_files/invalid_march.txt"

#### Using Panda to load the excel file. We are interested in only url column for now.

In [3]:
df = pd.read_excel(micro_sample_cvs)
urls = df["URL"]
school_names = df["School name"]
# print(urls)

### Helper Functions

In [12]:
def check(url):
    """ Helper function, check if url is a valid list <- our backup plan
    This functions helps to check the url that has service unavailable issues
    Since status code fails to check this."""
    
    try:
        urlopen(url)
        
    except urllib.error.URLError:
        print(url + " :URLError")
        return False
    except urllib.error.HTTPError:
        print(url +' :HTTPError')
        return False
    except SocketError:
        print(url + 'SocketError')
        return False
    return True

def check_url(url):
    """This functions uses the status code to determine if the link is valid. This resolves
    the links that redirects and most cases of authentication problems"""
    code = "[no code collected]"
    if url == "":
        return False
    try:
        r = requests.get(url, auth=HTTPDigestAuth('user', 'pass'), headers= {'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36"})
        code = r.status_code
        #backup plan for service unavailable issues
        if code == 503:
            return check(url)
        if code < 400:
            return True   
    except:
        pass
    print(str(url) +" ---Error code: " + str(code))
    return False    

# just write str to file
def write_file(str, file_name):
    with open(file_name, "a") as text_file:
        text_file.write(str)
def reset_text_file(file_name):
    if os.path.exists(file_name):
            with open(file_name, "w") as text_file:
                text_file.write("")
                
def count_valid_links(list_of_names,list_of_links, valid_file, invalid_file):
    """we want to determine how many valid and invalid links"""
    count_success, count_fail = 0, 0
    valid, invalid = '', ''
    for i in range(0,len(list_of_links)):
        if check_url(list_of_links[i]):
            valid += str(list_of_names[i]) +"\t" +list_of_links[i]+ '\n'
            count_success +=1
        else:
            invalid += str(list_of_links[i]) + '\n'
            count_fail += 1
    write_file(valid, valid_file)
    write_file(invalid, invalid_file)
    return count_success, count_fail

In [13]:
reset_text_file(invalid)
reset_text_file(valid)
yes, no =count_valid_links(school_names, urls, valid, invalid)
print(no)

nan ---Error code: [no code collected]
nan ---Error code: [no code collected]
nan ---Error code: [no code collected]
3


In [6]:
check_url("http://www.appletonpublicmontessori.com/")
print(yes, no)

285 15


#### There are 16 invalid links and the rest are valid.

In [7]:
check("http://www.appletonpublicmontessori.com/")
# for link in soup.findAll('a', attrs={'href': re.compile("((http|ftp)s?://.*?)")}):

True

In [1]:
# html_page = requests.get("http://www.wattslearningcenter.org/")
# soup = BeautifulSoup(html_page.text)
# count = 0;

# for link in soup.findAll('a'):
#     pattern = re.compile("((http|ftp)s?://.*?)")
#     url_parent = "http://www.wattslearningcenter.org/"
#     count +=1
#     current_link = link.get('href')
#     if not pattern.match(current_link):
#         current_link = urljoin(url_parent, current_link)
#     print(current_link)
# print(count)